#EEL891 - 2021.02 - Trabalho 1

##Classificação: sistema de apoio à decisão p/ aprovação de crédito

Matheus Lomba de Rezende Conde

DRE: 117085216

##Preparando o ambiente

###Importando as bibliotecas necessárias



In [31]:
#Bibliotecas de análise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

import io #Leitura do csv importado

In [ ]:
#Biblioteca para importação das bases de dados
from google.colab import files
uploaded = files.upload()

Saving conjunto_de_teste.csv to conjunto_de_teste (1).csv
Saving conjunto_de_treinamento.csv to conjunto_de_treinamento (1).csv


###Carregando o dataset

In [32]:
#Carga do dataset de treinamento
df_treino = pd.read_csv(io.BytesIO(uploaded['conjunto_de_treinamento.csv']))

#Carga do dataset de teste
df_teste = pd.read_csv(io.BytesIO(uploaded['conjunto_de_teste.csv']))

##Analisando o Dataset

###Número de Registros e Atributos nos datasets

In [ ]:
print(f'Shape treino: {df_treino.shape}')
print(f'Shape teste: {df_teste.shape}')

###Amostra do Dataset

In [53]:
df_treino.head().T

,0,1,2,3,4
produto_solicitado,1.0,1.0,1.0,1.0,7.0
dia_vencimento,10.0,25.0,20.0,20.0,1.0
tipo_endereco,1.0,1.0,1.0,1.0,1.0
idade,85.0,38.0,37.0,37.0,51.0
estado_civil,2.0,1.0,2.0,1.0,1.0
qtde_dependentes,0.0,0.0,0.0,1.0,3.0
nacionalidade,1.0,1.0,1.0,1.0,1.0
possui_telefone_residencial,1.0,1.0,1.0,1.0,1.0
tipo_residencia,1.0,1.0,5.0,1.0,0.0
meses_na_residencia,12.0,5.0,1.0,1.0,1.0


###Matriz de Correlação

In [ ]:
#Seta o tamanho de plotagem
sns.set(rc={'figure.figsize':(150,150)})
sns.set_context(context='paper',font_scale=7)
#Plota a matriz de correlação
sns.heatmap(df_treino.loc[:, df_treino.columns != 'SalePrice'].corr(), annot = True, cmap='nipy_spectral', vmin=-1, vmax=1)

###Analisando atributos com valores vazios

In [ ]:
plt.figure(figsize=(25,10))
sns.set_context('talk')

hm = sns.heatmap(df_treino.isnull(),yticklabels=False,cbar=False)
hm.set_yticklabels(labels=hm.get_yticklabels(), va='center')

Calculando a porcentagem de valores vazios em cada coluna:

In [ ]:
#Troca valores " " por Nulo
df_treino = df_treino.replace(" ", np.nan)

#Calcula o número de valores nulos ou " " em cada coluna
num_nulos = df_treino.isna().sum()
#Filtro - aparecer apenas colunas com nulos
num_nulos = num_nulos[num_nulos > 0]

#Calcula o percentual de nulos em cada coluna
num_linhas = df_treino.shape[0]
perc_nulos = num_nulos.apply(lambda x:(x/num_linhas)*100)
decrescente_perc_nulos = pd.Series(perc_nulos).sort_values(ascending=False)
decrescente_perc_nulos

###Variáveis Categóricas e Cardinalidade

In [ ]:
df_treino.dtypes

####Identificando Variáveis Categóricas:

In [ ]:
variaveis_categoricas = [
  x for x in df_treino.columns if df_treino[x].dtypes=="object"
]
print(variaveis_categoricas)

####Verificando a Cardinalidade das Variáveis Categóricas:

In [ ]:
for v in variaveis_categoricas:
  print("\n%15s:"%v, "%4d categorias" % len(df_treino[v].unique()))
  print(df_treino[v].unique())

forma_envio_solicitacao --> não-ordinal com 3 categorias **-> one-hot encoding**

sexo --> não-ordinal com 4 categorias **-> tratamento do nulo e one-hot encoding**

estado_onde_nasceu --> não-ordinal com 28 categorias **-> descartar**

estado_onde_reside --> não-ordinal com 27 categorias **-> descartar**

possui_telefone_residencial --> binária **-> binarizar (mapear para 0 ou 1)**

codigo_area_telefone_residencial --> não-ordinal com 81 categorias **-> descartar**

possui_telefone_celular -> binária  **-> binarizar (mapear para 0 ou 1)**

vinculo_formal_com_empresa --> binária **-> binarizar (mapear para 0 ou 1)**

estado_onde_trabalha --> não-ordinal com 28 categorias **-> descartar**

possui_telefone_trabalho --> binária **-> binarizar (mapear para 0 ou 1)**

codigo_area_telefone_trabalho --> não-ordinal com 77 categorias **-> descartar**

##Tratamento do Dataset

###Exclusão de atributos desnecessários

In [33]:
#Decisão feita com base no dicionário de dados fornecido no Kaggle e a Matriz de Correlação acima.
del df_treino["id_solicitante"] #Dicionário
del df_treino["possui_telefone_celular"] #Apenas valor N
del df_treino["possui_carro"] #Matriz de Correlação
del df_treino["grau_instrucao"] #Coluna totalmente vazia
del df_treino["meses_no_trabalho"] #Grande maioria da coluna vazia

#Remoção baseada na cardinalidade das variáveis categóricas
del df_treino["estado_onde_nasceu"]
del df_treino["estado_onde_reside"]
del df_treino["codigo_area_telefone_residencial"]
del df_treino["estado_onde_trabalha"]
del df_treino["codigo_area_telefone_trabalho"]

In [ ]:
df_treino.columns

Index(['produto_solicitado', 'dia_vencimento', 'forma_envio_solicitacao',
       'tipo_endereco', 'sexo', 'idade', 'estado_civil', 'qtde_dependentes',
       'nacionalidade', 'possui_telefone_residencial', 'tipo_residencia',
       'meses_na_residencia', 'possui_email', 'renda_mensal_regular',
       'renda_extra', 'possui_cartao_visa', 'possui_cartao_mastercard',
       'possui_cartao_diners', 'possui_cartao_amex', 'possui_outros_cartoes',
       'qtde_contas_bancarias', 'qtde_contas_bancarias_especiais',
       'valor_patrimonio_pessoal', 'vinculo_formal_com_empresa',
       'possui_telefone_trabalho', 'profissao', 'ocupacao',
       'profissao_companheiro', 'grau_instrucao_companheiro',
       'local_onde_reside', 'local_onde_trabalha', 'inadimplente'],
      dtype='object')

###Tratamento - Colunas com valor nulo

Observando-se novamente os atributos com valores vazios:

In [34]:
#Recalculando o a porcentagem de nulos
df_treino = df_treino.replace(" ", np.nan)
num_nulos = df_treino.isna().sum()
num_nulos = num_nulos[num_nulos > 0]
num_linhas = df_treino.shape[0]
perc_nulos = num_nulos.apply(lambda x:(x/num_linhas)*100)
decrescente_perc_nulos = pd.Series(perc_nulos).sort_values(ascending=False)
decrescente_perc_nulos

grau_instrucao_companheiro    64.300
profissao_companheiro         57.570
profissao                     15.485
ocupacao                      14.890
meses_na_residencia            7.250
tipo_residencia                2.680
sexo                           0.035
dtype: float64

Levando-se em consideração que existem diversas colunas com mais de 50% de valores nulos e que, devido à pouca quantidade não serão de bom uso para o estudo, elas serão removidas da base.

In [35]:
for p in decrescente_perc_nulos:
  if p >= 50:
    del df_treino[decrescente_perc_nulos[decrescente_perc_nulos == p].index[0]]

Já em relação aos outros atributos listados, será utilizada a média (arredondada para inteiros) para preencher os campos nulos das variáveis numéricas e um valor neutro para o preenchimento das variáveis categóricas.

In [36]:
for p in decrescente_perc_nulos:
  if p < 50:
    atributo = decrescente_perc_nulos[decrescente_perc_nulos == p].index[0]
    print(atributo, " - ", df_treino[atributo].dtypes)

profissao  -  float64
ocupacao  -  float64
meses_na_residencia  -  float64
tipo_residencia  -  float64
sexo  -  object


In [37]:
#Preenchimento dos campos nulos em "sexo" por "N":
df_treino[["sexo"]] = df_treino[["sexo"]].fillna(value="N")

In [38]:
#Preenchimento dos campos nulos em "profissao", "ocupacao", "meses_na_residencia" e "tipo_residencia" pela média respectiva.
for p in decrescente_perc_nulos:
  atributo = decrescente_perc_nulos[decrescente_perc_nulos == p].index[0]
  if p < 50 and atributo != "sexo":
    df_treino[[atributo]] = df_treino[[atributo]].fillna(value=df_treino[[atributo]].mean().round())

In [39]:
df_treino.describe()

,produto_solicitado,dia_vencimento,tipo_endereco,idade,estado_civil,qtde_dependentes,nacionalidade,tipo_residencia,meses_na_residencia,possui_email,...,possui_cartao_amex,possui_outros_cartoes,qtde_contas_bancarias,qtde_contas_bancarias_especiais,valor_patrimonio_pessoal,profissao,ocupacao,local_onde_reside,local_onde_trabalha,inadimplente
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,2.000000e+04,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,1.284350,13.147250,1.006350,42.352500,2.120850,0.666400,0.961600,1.25430,9.603450,0.799200,...,0.002050,0.002250,0.361100,0.361100,2.095614e+03,8.038100,2.602800,581.295250,581.295250,0.500000
std,1.008239,6.748507,0.079435,14.930177,1.332004,1.236725,0.202305,0.87268,10.256849,0.400609,...,0.045232,0.047382,0.481164,0.481164,4.403344e+04,2.951776,1.423774,227.369798,227.369798,0.500013
min,1.000000,1.000000,1.000000,7.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,105.000000,105.000000,0.000000
25%,1.000000,10.000000,1.000000,31.000000,1.000000,0.000000,1.000000,1.00000,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,8.000000,2.000000,444.000000,444.000000,0.000000
50%,1.000000,10.000000,1.000000,40.000000,2.000000,0.000000,1.000000,1.00000,7.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,9.000000,2.000000,596.000000,596.000000,0.500000
75%,1.000000,20.000000,1.000000,52.000000,2.000000,1.000000,1.000000,1.00000,13.000000,1.000000,...,0.000000,0.000000,1.000000,1.000000,0.000000e+00,9.000000,4.000000,728.000000,728.000000,1.000000
max,7.000000,25.000000,2.000000,106.000000,7.000000,53.000000,2.000000,5.00000,228.000000,1.000000,...,1.000000,1.000000,2.000000,2.000000,6.000000e+06,17.000000,5.000000,999.000000,999.000000,1.000000


Calculando novamente a porcentagem de nulos:

In [40]:
#Calcula o número de valores nulos em cada coluna
num_nulos = df_treino.isna().sum()
#Filtro - aparecer apenas colunas com nulos
num_nulos = num_nulos[num_nulos > 0]

#Calcula o percentual de nulos em cada coluna
num_linhas = df_treino.shape[0]
perc_nulos = num_nulos.apply(lambda x:(x/num_linhas)*100)
decrescente_perc_nulos = pd.Series(perc_nulos).sort_values(ascending=False)
decrescente_perc_nulos

#retorno vazio

Series([], dtype: int64)

In [41]:
df_treino.head()

,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,nacionalidade,possui_telefone_residencial,...,qtde_contas_bancarias,qtde_contas_bancarias_especiais,valor_patrimonio_pessoal,vinculo_formal_com_empresa,possui_telefone_trabalho,profissao,ocupacao,local_onde_reside,local_onde_trabalha,inadimplente
0,1,10,presencial,1,M,85,2,0,1,Y,...,1,1,0.0,N,N,9.0,1.0,600.0,600.0,0
1,1,25,internet,1,F,38,1,0,1,Y,...,0,0,0.0,N,N,2.0,5.0,492.0,492.0,0
2,1,20,internet,1,F,37,2,0,1,Y,...,0,0,0.0,N,N,8.0,3.0,450.0,450.0,1
3,1,20,internet,1,M,37,1,1,1,Y,...,0,0,0.0,Y,Y,9.0,2.0,932.0,932.0,1
4,7,1,internet,1,F,51,1,3,1,Y,...,0,0,0.0,Y,N,9.0,5.0,440.0,440.0,1


###Tratamento das Variáveis Categóricas

In [ ]:
#Printando as colunas até agora:
print(df_treino.head(3).T)

                                          0         1         2
produto_solicitado                        1         1         1
dia_vencimento                           10        25        20
forma_envio_solicitacao          presencial  internet  internet
tipo_endereco                             1         1         1
sexo                                      M         F         F
idade                                    85        38        37
estado_civil                              2         1         2
qtde_dependentes                          0         0         0
nacionalidade                             1         1         1
possui_telefone_residencial               Y         Y         Y
tipo_residencia                         1.0       1.0       5.0
meses_na_residencia                    12.0       5.0       1.0
possui_email                              0         1         1
renda_mensal_regular                  480.0     380.0     600.0
renda_extra                             

In [42]:
#Aplicando one-hot encoding nas variáveis "sexo" e "forma_envio_solicitacao":
df_treino = pd.get_dummies(df_treino, columns=["sexo", "forma_envio_solicitacao"])
print(df_treino.head(3).T)

                                        0      1      2
produto_solicitado                      1      1      1
dia_vencimento                         10     25     20
tipo_endereco                           1      1      1
idade                                  85     38     37
estado_civil                            2      1      2
qtde_dependentes                        0      0      0
nacionalidade                           1      1      1
possui_telefone_residencial             Y      Y      Y
tipo_residencia                       1.0    1.0    5.0
meses_na_residencia                  12.0    5.0    1.0
possui_email                            0      1      1
renda_mensal_regular                480.0  380.0  600.0
renda_extra                           0.0    0.0    0.0
possui_cartao_visa                      0      0      0
possui_cartao_mastercard                0      0      0
possui_cartao_diners                    0      0      0
possui_cartao_amex                      0      0

In [43]:
#Aplicando a binarização:
binarizador = LabelBinarizer()

for v in ["possui_telefone_residencial", "vinculo_formal_com_empresa", "possui_telefone_trabalho"]:
  df_treino[v] = binarizador.fit_transform(df_treino[v])

print(df_treino.head(3).T)

                                        0      1      2
produto_solicitado                    1.0    1.0    1.0
dia_vencimento                       10.0   25.0   20.0
tipo_endereco                         1.0    1.0    1.0
idade                                85.0   38.0   37.0
estado_civil                          2.0    1.0    2.0
qtde_dependentes                      0.0    0.0    0.0
nacionalidade                         1.0    1.0    1.0
possui_telefone_residencial           1.0    1.0    1.0
tipo_residencia                       1.0    1.0    5.0
meses_na_residencia                  12.0    5.0    1.0
possui_email                          0.0    1.0    1.0
renda_mensal_regular                480.0  380.0  600.0
renda_extra                           0.0    0.0    0.0
possui_cartao_visa                    0.0    0.0    0.0
possui_cartao_mastercard              0.0    0.0    0.0
possui_cartao_diners                  0.0    0.0    0.0
possui_cartao_amex                    0.0    0.0

####Verificando o tipo de todas as colunas novamente

In [ ]:
df_treino.dtypes

##Análise Exploratória

###Atributo "Inadimplentes" VS outros atributos

In [ ]:
#Verificando a quantidade de inadimplentes (1) ou bom pagador (0)
df_treino["inadimplente"].value_counts()

In [ ]:
#Verificando a relação entre Inadimplente e outros atributos
grupo = df_treino.groupby(["inadimplente"]).mean()
grupo.T

In [ ]:
#Identificando quais atributos tem Média com maior diferença entre Bom Pagador e Inadimplente
listaAtributos = grupo.columns.values.tolist()
dictImpacto = {}

for v in range(0, len(listaAtributos)):
  dictImpacto[listaAtributos[v]] = abs(grupo.T[0][v] - grupo.T[1][v])/((grupo.T[0][v] + grupo.T[1][v])/2)

dictImpacto_ordenado = sorted(dictImpacto.items(), key=lambda x: x[1], reverse=True)
print(dictImpacto_ordenado)

###Plotar diagrama de dispersão

In [ ]:
atr1 = "idade"
atr2 = "renda_extra"

#Inadimplente = vermelho e Bom pagador = azul
cores = [ 'red' if x else 'blue' for x in df_treino['inadimplente'] ]

grafico = df_treino.plot.scatter(
    atr1,
    atr2,
    c = cores,
    s = 5,
    alpha = 0.5,
    figsize = (10,10)
)

plt.show()

Retirando a linha que possui o outlier de renda_extra do gráfico de dispersão:

In [ ]:
#Novo dataset criado sem o outlier, apenas para melhorar a visualização do gráfico de dispersão
df_dispersao = df_treino[df_treino.renda_extra != df_treino['renda_extra'].max()]
#print(df_dispersao["renda_extra"].describe())

In [ ]:
atr1 = "idade"
atr2 = "renda_extra"

#Inadimplente = vermelho e Bom pagador = azul
cores = [ 'red' if x else 'blue' for x in df_dispersao['inadimplente'] ]

grafico = df_dispersao.plot.scatter(
    atr1,
    atr2,
    c = cores,
    s = 10,
    alpha = 0.5,
    figsize = (15,10)
)

##Classificação

###Tratamento - Dataset Testes

In [44]:
#Decisão feita com base no dicionário de dados fornecido no Kaggle e a Matriz de Correlação acima.
#del df_teste["id_solicitante"] #Dicionário
del df_teste["possui_telefone_celular"] #Apenas valor N
del df_teste["possui_carro"] #Matriz de Correlação
del df_teste["grau_instrucao"] #Coluna totalmente vazia
del df_teste["meses_no_trabalho"] #Grande maioria da coluna vazia

#Remoção baseada na cardinalidade das variáveis categóricas
del df_teste["estado_onde_nasceu"]
del df_teste["estado_onde_reside"]
del df_teste["codigo_area_telefone_residencial"]
del df_teste["estado_onde_trabalha"]
del df_teste["codigo_area_telefone_trabalho"]

#Recalculando o a porcentagem de nulos
df_teste = df_teste.replace(" ", np.nan)
num_nulos = df_teste.isna().sum()
num_nulos = num_nulos[num_nulos > 0]
num_linhas = df_teste.shape[0]
perc_nulos = num_nulos.apply(lambda x:(x/num_linhas)*100)
decrescente_perc_nulos = pd.Series(perc_nulos).sort_values(ascending=False)
decrescente_perc_nulos

for p in decrescente_perc_nulos:
  if p >= 50:
    del df_teste[decrescente_perc_nulos[decrescente_perc_nulos == p].index[0]]

for p in decrescente_perc_nulos:
  if p < 50:
    atributo = decrescente_perc_nulos[decrescente_perc_nulos == p].index[0]

#Preenchimento dos campos nulos em "sexo" por "N":
df_teste[["sexo"]] = df_teste[["sexo"]].fillna(value="N")

#Preenchimento dos campos nulos em "profissao", "ocupacao", "meses_na_residencia" e "tipo_residencia" pela média respectiva.
for p in decrescente_perc_nulos:
  atributo = decrescente_perc_nulos[decrescente_perc_nulos == p].index[0]
  if p < 50 and atributo != "sexo":
    df_teste[[atributo]] = df_teste[[atributo]].fillna(value=df_teste[[atributo]].mean().round())

#Aplicando one-hot encoding nas variáveis "sexo" e "forma_envio_solicitacao":
df_teste = pd.get_dummies(df_teste, columns=["sexo", "forma_envio_solicitacao"])

#Aplicando a binarização:
binarizador = LabelBinarizer()
for v in ["possui_telefone_residencial", "vinculo_formal_com_empresa", "possui_telefone_trabalho"]:
  df_teste[v] = binarizador.fit_transform(df_teste[v])


#Criando uma coluna Inadimplentes no dataset de teste
df_teste["inadimplente"] = np.nan

In [45]:
df_teste.head().T

,0,1,2,3,4
id_solicitante,20001.0,20002.00,20003.0,20004.0,20005.0
produto_solicitado,1.0,1.00,1.0,1.0,1.0
dia_vencimento,25.0,10.00,10.0,10.0,10.0
tipo_endereco,1.0,1.00,1.0,1.0,1.0
idade,37.0,31.00,18.0,55.0,55.0
estado_civil,2.0,2.00,2.0,2.0,1.0
qtde_dependentes,0.0,0.00,0.0,0.0,0.0
nacionalidade,1.0,1.00,1.0,1.0,1.0
possui_telefone_residencial,1.0,1.00,1.0,1.0,1.0
tipo_residencia,1.0,1.00,1.0,1.0,1.0


###Selecionar os atributos que serão usados pelo classificador:

In [ ]:
#Lista de Colunas para usar a seguir
df_treino.columns

In [46]:
atributos_selecionados = [
       'produto_solicitado', 
       'dia_vencimento', 
       #'tipo_endereco', 
       'idade',
       'estado_civil', 
       'qtde_dependentes', 
       #'nacionalidade',
       'possui_telefone_residencial', 
       'tipo_residencia', 
       'meses_na_residencia',
       #'possui_email', 
       'renda_mensal_regular', 
       'renda_extra',
       #'possui_cartao_visa', 
       'possui_cartao_mastercard',
       #'possui_cartao_diners', 
       #'possui_cartao_amex', 
       #'possui_outros_cartoes',
       #'qtde_contas_bancarias', 
       #'qtde_contas_bancarias_especiais',
       'valor_patrimonio_pessoal', 
       #'vinculo_formal_com_empresa',
       'possui_telefone_trabalho', 
       #'profissao', 
       'ocupacao',
       'local_onde_reside', 
       'local_onde_trabalha', 
       'sexo_F',
       'sexo_M', 
       #'sexo_N', 
       #'forma_envio_solicitacao_correio',
       #'forma_envio_solicitacao_internet',
       #'forma_envio_solicitacao_presencial',
       'inadimplente'
]

df_treino1 = df_treino[atributos_selecionados]
df_teste1 = df_teste[atributos_selecionados]

###Criando arrays X e Y para separar os Atributos e o Alvo

In [47]:
#Montando base de treino e base de teste
df_emb = df_treino1.sample(frac=1, random_state=12345)
x = df_emb.loc[:, df_emb.columns != "inadimplente"].values
y = df_emb.loc[:, df_emb.columns == "inadimplente"].values

#Definindo conjunto de treino e de teste - como já estão separados, só será realizada a divisão entre x e y
q = 15000
#Treino
x_treino = x[:q,:]
y_treino = y[:q].ravel()

#Teste
x_teste = x[q:,:]
y_teste = y[q:].ravel()

#Avaliação
x_avaliacao = df_teste1.loc[:, df_teste1.columns != "inadimplente"].values
#y_avaliacao = df_teste1.loc[:, df_teste1.columns == "inadimplente"].values.ravel()

###Normalizando o Dataset - MinMaxScaler

In [ ]:
ajuste = MinMaxScaler()
ajuste.fit(x_treino)

x_treino = ajuste.transform(x_treino)
x_teste = ajuste.transform(x_teste)
x_avaliacao = ajuste.transform(x_avaliacao)

###Treinando um classificador KNN

In [ ]:
classificador = KNeighborsClassifier(n_neighbors = 10)
classificador = classificador.fit(x_treino, y_treino)

####Obtendo as respostas do classificador no conjunto de treino

In [ ]:
y_resposta_treino = classificador.predict(x_treino)

####Obtendo as respostas do classificador no conjunto de teste

In [ ]:
y_resposta_teste = classificador.predict(x_teste)

####Verificando a acurácia do classificador

In [ ]:
#Acurácia na amostra de treino
total = len(y_treino)
acertos = sum(y_resposta_treino == y_treino)
erros = sum(y_resposta_treino != y_treino)

print("Total de amostras: ", total)
print("Respostas corretas: ", acertos)
print("Respostas erradas: ", erros)

acuracia = acertos/total

print("Acurácia = %.1f %%" % (100*acuracia))

In [ ]:
#Acurácia na amostra de teste
total = len(y_teste)
acertos = sum(y_resposta_teste == y_teste)
erros = sum(y_resposta_teste != y_teste)

print("Total de amostras: ", total)
print("Respostas corretas: ", acertos)
print("Respostas erradas: ", erros)

acuracia = acertos/total

print("Acurácia = %.1f %%" % (100*acuracia))

###Análise - Variação da Acurácia

As células estão ordenadas por melhor resultado 

In [ ]:
for k in range(51, 100, 2):
  classificador = KNeighborsClassifier(
      n_neighbors = k,
      weights = 'distance',
      p = 1
  )
  classificador = classificador.fit(x_treino, y_treino)

  y_resposta_treino = classificador.predict(x_treino)
  y_resposta_teste = classificador.predict(x_teste)

  acuracia_treino = sum(y_resposta_treino == y_treino)/len(y_treino)
  acuracia_teste = sum(y_resposta_teste == y_teste)/len(y_teste)

  print(
      "%3d" % k,
      "%6.1f" % (100*acuracia_treino),
      "%6.1f" % (100*acuracia_teste)
  )

  1  100.0   52.2
  3   75.9   51.8
  5   69.5   52.0
  7   67.0   52.6
  9   65.1   52.6
 11   63.6   52.1
 13   62.7   52.4
 15   62.1   51.8
 17   61.4   51.7
 19   61.0   52.6
 21   60.3   52.0
 23   60.1   52.5


In [ ]:
for k in range(1, 50, 2):
  classificador = KNeighborsClassifier(
      n_neighbors = k,
      weights = 'distance'
  )
  classificador = classificador.fit(x_treino, y_treino)

  y_resposta_treino = classificador.predict(x_treino)
  y_resposta_teste = classificador.predict(x_teste)

  acuracia_treino = sum(y_resposta_treino == y_treino)/len(y_treino)
  acuracia_teste = sum(y_resposta_teste == y_teste)/len(y_teste)

  print(
      "%3d" % k,
      "%6.1f" % (100*acuracia_treino),
      "%6.1f" % (100*acuracia_teste)
  )

In [ ]:
for k in range(1, 50, 2):
  classificador = KNeighborsClassifier(
      n_neighbors = k,
      weights = 'uniform',
      p = 1
  )
  classificador = classificador.fit(x_treino, y_treino)

  y_resposta_treino = classificador.predict(x_treino)
  y_resposta_teste = classificador.predict(x_teste)

  acuracia_treino = sum(y_resposta_treino == y_treino)/len(y_treino)
  acuracia_teste = sum(y_resposta_teste == y_teste)/len(y_teste)

  print(
      "%3d" % k,
      "%6.1f" % (100*acuracia_treino),
      "%6.1f" % (100*acuracia_teste)
  )

In [ ]:
for k in range(1, 50, 2):
  classificador = KNeighborsClassifier(
      n_neighbors = k
  )
  classificador = classificador.fit(x_treino, y_treino)

  y_resposta_treino = classificador.predict(x_treino)
  y_resposta_teste = classificador.predict(x_teste)

  acuracia_treino = sum(y_resposta_treino == y_treino)/len(y_treino)
  acuracia_teste = sum(y_resposta_teste == y_teste)/len(y_teste)

  print(
      "%3d" % k,
      "%6.1f" % (100*acuracia_treino),
      "%6.1f" % (100*acuracia_teste)
  )

###Análise - Validação Cruzada

In [55]:
for k in range(81, 120, 2):
  classificador = KNeighborsClassifier(
      n_neighbors = k,
      weights = 'distance',
      p = 1
  )

  scores = cross_val_score(
      classificador,
      x,
      y.ravel(),
      cv=8
  )

  print(
      "k = %2d" % k,
      "scores = ", scores,
      "acurácia média = %6.1f" % (100*sum(scores)/8)
  )

k = 81 scores =  [0.5408 0.542  0.52   0.5444 0.524  0.5452 0.5332 0.5464] acurácia média =   53.7
k = 83 scores =  [0.5452 0.5436 0.5212 0.5448 0.5228 0.5464 0.5368 0.5456] acurácia média =   53.8
k = 85 scores =  [0.5452 0.5464 0.5248 0.5436 0.5236 0.5456 0.5372 0.542 ] acurácia média =   53.9
k = 87 scores =  [0.5496 0.5444 0.5256 0.5432 0.5256 0.5472 0.536  0.5396] acurácia média =   53.9
k = 89 scores =  [0.546  0.5456 0.5284 0.5476 0.5272 0.5432 0.5348 0.54  ] acurácia média =   53.9
k = 91 scores =  [0.548  0.5456 0.5244 0.55   0.53   0.5416 0.536  0.5376] acurácia média =   53.9
k = 93 scores =  [0.5492 0.5448 0.5248 0.5476 0.5284 0.5432 0.5396 0.5396] acurácia média =   54.0
k = 95 scores =  [0.5504 0.5444 0.5236 0.548  0.5264 0.5488 0.54   0.536 ] acurácia média =   54.0
k = 97 scores =  [0.5492 0.5428 0.5264 0.5456 0.5256 0.5492 0.5404 0.534 ] acurácia média =   53.9
k = 99 scores =  [0.5484 0.5452 0.5264 0.5488 0.5268 0.5444 0.5376 0.5328] acurácia média =   53.9
k = 101 sc

###Predição dos Resultados

Pela avaliação dos modelos acima, ficou decidido que os melhores hiperparâmetros são:

k = 93;

weights = "distance";

p = 1;

In [ ]:
classificador = KNeighborsClassifier(
  n_neighbors = 93,
  weights = "distance",
  p = 1
)

classificador = classificador.fit(x_treino, y_treino)

y_resposta = classificador.predict(x_avaliacao)

'''for x in range(0, len(y_resposta)):
  print(y_resposta[x])'''

print (y_resposta)

In [ ]:
df_teste["inadimplente"] = y_resposta

In [ ]:
df_teste.head(20).T

####Resultado Final

In [ ]:
resultado_final = df_teste[["id_solicitante", "inadimplente"]]
resultado_final

In [ ]:
resultado_final.to_csv("resultado_T1.csv", index=False)

In [ ]:
from google.colab import files
files.download("resultado_T1.csv")